# 🧠 评测数据相关性分析（Notebook 模板）

本 Notebook 基于你的数据规范，按模块完成：数据读取 → 字段派生 → 基础分析（3项）→ 深入分析（6项）→ 汇总结论。

- 输入：CSV 文件，必含字段：`evaluator_id, seq_no, intent_content, left_candidate_content, left_application_name, right_candidate_content, right_application_name, time_spent_sec, winner, left_application_count, right_candidate_count`
- 依赖：pandas, numpy, scipy, statsmodels, matplotlib, seaborn
- 缺失值将自动忽略，统计检验报告关键指标（p 值），每节输出一句话结论。

运行顺序：从上到下依次运行即可。


In [ ]:
# 安装/导入依赖 & 基本设置
import sys, os, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'Heiti TC', 'PingFang SC', 'Microsoft YaHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
sns.set(style='whitegrid', palette='Set2')

print('Versions:',
      'pandas', pd.__version__,
      'numpy', np.__version__,
      'scipy', stats.__version__ if hasattr(stats, '__version__') else 'n/a',
      'statsmodels', sm.__version__,
      'seaborn', sns.__version__)


In [ ]:
# 读取 CSV（请修改为你的实际路径）
CSV_PATH = 'data.csv'  # TODO: 修改为你的文件路径

usecols = [
    'evaluator_id','seq_no','intent_content',
    'left_candidate_content','left_application_name','left_application_count',
    'right_candidate_content','right_application_name','right_candidate_count',
    'time_spent_sec','winner'
]

df = pd.read_csv(CSV_PATH, usecols=usecols)
print('Raw shape:', df.shape)
print(df.head(2))


In [ ]:
# 字段派生（见规范）

df2 = df.copy()

# 胜出位置
df2['winner_side'] = np.where(df2['winner'].eq(df2['left_application_name']), 'left',
                        np.where(df2['winner'].eq(df2['right_application_name']), 'right', pd.NA))
# 失败方模型
df2['loser_application_name'] = np.where(df2['winner_side'].eq('left'), df2['right_application_name'],
                                   np.where(df2['winner_side'].eq('right'), df2['left_application_name'], pd.NA))
# 胜/负字数
df2['winner_len'] = np.where(df2['winner_side'].eq('left'), df2['left_application_count'],
                        np.where(df2['winner_side'].eq('right'), df2['right_candidate_count'], pd.NA))

df2['loser_len'] = np.where(df2['winner_side'].eq('left'), df2['right_candidate_count'],
                       np.where(df2['winner_side'].eq('right'), df2['left_application_count'], pd.NA))

# 字数差（左-右）
df2['len_diff'] = df2['left_application_count'] - df2['right_candidate_count']

# 模型对模型（按字母序）
df2['pair_model'] = df2.apply(lambda r: '|'.join(sorted([str(r['left_application_name']), str(r['right_application_name'])])), axis=1)

# 左是否胜出
df2['left_win'] = (df2['winner_side'] == 'left').astype('Int64')

# 答题时长分箱
bins = [-np.inf, 3, 8, 20, np.inf]
labels = ['very_fast','fast','normal','slow']
df2['time_bin'] = pd.cut(df2['time_spent_sec'], bins=bins, labels=labels)

print('Derived shape:', df2.shape)
df2.head(3)


In [ ]:
# 第一部分：基础分析（3项）

# 1) 模型偏好分析：总体胜率 & 评测人偏好
win_by_model = df2.groupby('winner')[['seq_no']].count().rename(columns={'seq_no':'wins'}).reset_index()
win_by_model['win_rate'] = win_by_model['wins'] / len(df2)
print('模型总体胜率：')
print(win_by_model.sort_values('win_rate', ascending=False).head(10))

plt.figure(figsize=(8,4))
sns.barplot(data=win_by_model, x='winner', y='win_rate', color='#6366F1')
plt.title('模型总体胜率')
plt.ylabel('胜率')
plt.xlabel('模型')
plt.xticks(rotation=30, ha='right')
plt.show()

# 评测人是否偏爱某模型（以其投票中对该模型的比例>期望？可做卡方/二项）
# 这里示例：对每个 evaluator 统计其最常投的模型
pref_by_eval = df2.groupby(['evaluator_id','winner']).size().unstack(fill_value=0)
pref_top = pref_by_eval.div(pref_by_eval.sum(1), 0).max(1)
print('评测人最大模型偏好比例（前10）:\n', pref_top.sort_values(ascending=False).head(10))

# 2) 位置偏好分析（左 vs 右）
left_win_rate = df2['left_win'].mean()
print(f"整体左边胜出比例：{left_win_rate:.3f}")

# 对每个评测人做二项检验（p=0.5）
from statsmodels.stats.proportion import binom_test
res_list = []
for uid, g in df2.groupby('evaluator_id'):
    n = g['left_win'].notna().sum()
    k = g['left_win'].sum()
    if n > 0:
        pval = binom_test(k, n, 0.5, alternative='two-sided')
        res_list.append({'evaluator_id': uid, 'n': n, 'left_win_rate': k/n, 'p_value': pval})
res_df = pd.DataFrame(res_list).sort_values('p_value')
print('显著位置偏好评测人（p<0.05）数量：', (res_df['p_value']<0.05).sum())

plt.figure(figsize=(6,4))
sns.histplot(df2['left_win'], bins=2)
plt.title('左/右胜出比例')
plt.xlabel('left_win (1=左胜)')
plt.show()

# 3) 答案长度影响：胜者是否更长？相关性
len_diff_valid = df2.dropna(subset=['winner_len','loser_len'])
len_delta = (len_diff_valid['winner_len'] - len_diff_valid['loser_len']).dropna()
print('胜者-败者字数差：均值', len_delta.mean(), '中位数', len_delta.median())

plt.figure(figsize=(7,4))
sns.histplot(len_delta, bins=40)
plt.title('胜者字数-败者字数 分布')
plt.show()

# 相关性（字数差 vs 左边胜出）
valid_corr = df2.dropna(subset=['len_diff','left_win'])
pear = valid_corr['len_diff'].corr(valid_corr['left_win'], method='pearson')
spea = valid_corr['len_diff'].corr(valid_corr['left_win'], method='spearman')
print(f"Pearson: {pear:.3f}, Spearman: {spea:.3f}")
print('结论：字数差与胜负存在{}相关关系'.format('一定程度的' if abs(pear)>0.2 or abs(spea)>0.2 else '较弱/不明显'))


In [ ]:
# 第二部分：深入分析（6项）

# 4) 评测人偏好诊断（位置/模型）
by_eval = df2.groupby('evaluator_id').agg(
    n=('seq_no','count'),
    left_rate=('left_win','mean')
)
model_pref = df2.groupby(['evaluator_id','winner']).size().groupby(level=0).apply(lambda s: (s/s.sum()).max())
by_eval['model_top_rate'] = model_pref
biased_evals = by_eval[(by_eval['left_rate'].notna()) & ((by_eval['left_rate']<0.35) | (by_eval['left_rate']>0.65) | (by_eval['model_top_rate']>0.65))]
print('明显偏好评测人数量：', len(biased_evals))
biased_evals.head(10)

# 5) 模型对模型胜率矩阵（公平对战）
pairs = df2.dropna(subset=['winner_side'])
# 计算 pair 中左模型 vs 右模型的胜率，然后聚合到按字母序的 pair_model
pairs['left_model'] = pairs['left_application_name']
pairs['right_model'] = pairs['right_application_name']

# 左模型胜率（在这一对）
pair_stats = pairs.groupby(['left_model','right_model']).agg(
    n=('seq_no','count'),
    left_win_rate=('left_win','mean')
).reset_index()
# 转为对称：按字母序映射为同一个键
pair_stats['pair_key'] = pair_stats.apply(lambda r: '|'.join(sorted([str(r['left_model']), str(r['right_model'])])), axis=1)
agg = pair_stats.groupby('pair_key').agg(n=('n','sum'), win_rate=('left_win_rate','mean')).reset_index()
print('模型对模型样例：')
print(agg.head())

# 6) 长度与胜负的多变量分析：逻辑回归（左胜=1）
logit_df = df2.dropna(subset=['left_win','len_diff','left_application_name','right_application_name']).copy()
logit_df['left_win'] = logit_df['left_win'].astype(int)
model = smf.logit('left_win ~ len_diff + C(left_application_name) + C(right_application_name)', data=logit_df).fit(disp=False)
print(model.summary())

# 7) 答题时长的影响分析
by_bin = df2.dropna(subset=['time_bin','left_win']).groupby('time_bin')['left_win'].mean().reindex(['very_fast','fast','normal','slow'])
by_bin.plot(kind='bar', color='#8B5CF6', title='不同答题时长下左侧胜率')
plt.ylabel('左胜率')
plt.show()

# 给建议阈值（示例：very_fast<3s 可考虑排除）
print('建议：可考虑过滤 very_fast (<3s) 的记录，以降低偶然偏差影响。')

# 8) 数据清洗与对比
clean = df2.copy()
# 去掉显著偏左/右与过短记录
biased_ids = set(biased_evals.index)
clean = clean[~clean['evaluator_id'].isin(biased_ids)]
clean = clean[~(clean['time_spent_sec']<3)]

orig_left_rate = df2['left_win'].mean()
clean_left_rate = clean['left_win'].mean()
print(f'原始左胜率={orig_left_rate:.3f}, 清洗后左胜率={clean_left_rate:.3f}')

orig_model_win = df2.groupby('winner').size()/len(df2)
clean_model_win = clean.groupby('winner').size()/len(clean)
print('原始模型胜率Top5:\n', orig_model_win.sort_values(ascending=False).head())
print('清洗后模型胜率Top5:\n', clean_model_win.sort_values(ascending=False).head())

# 9) 按 intent 分析模型表现
intent_model_win = df2.dropna(subset=['winner']).groupby(['intent_content','winner']).size().groupby(level=0).apply(lambda s: s/s.sum()).reset_index(name='win_rate')
print(intent_model_win.head())

plt.figure(figsize=(10,5))
sample_intents = intent_model_win['intent_content'].dropna().unique()[:8]
sns.barplot(data=intent_model_win[intent_model_win['intent_content'].isin(sample_intents)], x='intent_content', y='win_rate', hue='winner')
plt.title('按 Intent 的模型胜率（示例Top8 Intent）')
plt.xticks(rotation=30, ha='right')
plt.show()

# 10)（选做）时间与质量的联合分析：这里给出方向性示例
# 可计算每位评测人的决策一致性（如对同模型/同意图投票的方差等）
# 这里示意：每人对 left_win 的方差
stab = df2.groupby('evaluator_id')['left_win'].var().dropna()
print('评测人决策稳定性（left_win 方差）样例：\n', stab.describe())
